In [12]:
import re
import json
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaModel
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Function to clean code
def clean_code(code):
    code = re.sub(r'\/\*[\s\S]*?\*\/', '', code)  # Remove multiline comments
    code = re.sub(r'\/\/.*', '', code)  # Remove single line comments
    code = re.sub(r'#.*', '', code)  # Remove python comments
    code = re.sub(r'\s*\n\s*', '\n', code)  # Remove extra whitespace around newlines
    return code.strip()

# Function to label code (0 for safe, 1 for unsafe)
def label_code(snippet):
    unsafe_patterns = {
        'eval': 'Code Injection',                        # Executes code from a string
        'innerHTML': 'Cross-Site Scripting (XSS)',       # Inserts HTML content into an element
        'outerHTML': 'Cross-Site Scripting (XSS)',       # Similar to innerHTML but for the element itself
        'document.write': 'Cross-Site Scripting (XSS)',  # Writes directly to the HTML document
        'setTimeout': 'Potential Code Injection',        # Executes code after a delay
        'setInterval': 'Potential Code Injection',       # Repeatedly executes code at intervals
        'Function': 'Code Injection',                    # Creates new functions from strings
        'location.href': 'Open Redirect',                # Changes the URL of the current page
        'document.location': 'Open Redirect',            # Similar to location.href
        'XMLHttpRequest': 'Sensitive Data Exposure',     # Handles HTTP requests, can be misused
        '<script>': 'Cross-Site Scripting (XSS)',        # Directly injects JavaScript code
        'document.body.innerHTML': 'Cross-Site Scripting (XSS)'  # Sets the HTML content of the body element
    }

    unsafe_lines = []
    for line in snippet.split('\n'):
        for pattern, vuln_type in unsafe_patterns.items():
            if pattern in line:
                unsafe_lines.append((line.strip(), vuln_type))
    return unsafe_lines

# Load pre-trained model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
model = RobertaModel.from_pretrained('microsoft/codebert-base')

# Load existing preprocessed dataset
with open('javascript.json', 'r') as f:
    data = json.load(f)


In [8]:
# Tokenize and get embeddings for the code snippets
inputs = tokenizer([d['code'] for d in data], padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)
embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

In [9]:
# Create labels
labels = np.array([d['label'] for d in data])

# Train a simple classifier
clf = RandomForestClassifier()
clf.fit(embeddings, labels)

# Testing the classifier with a new code snippet
test_code = """
    var userInput = '{"name": "John", "age": 30}';
    var userObj;
    try {
        userObj = JSON.parse(userInput);  // Safely parse JSON without using eval
    } catch (e) {
        console.error("Invalid JSON input");
    }

    var userInput = "alert('Hello, world!')";
    eval(userInput);  // Unsafe
    var safe = 1243;
    document.getElementById('output').innerHTML = userInput;  // Unsafe
    document.write(userInput);  // Unsafe


"""
cleaned_test_code = clean_code(test_code)
unsafe_lines = label_code(cleaned_test_code)
# unsafe_lines = label_code(test_code)

print("Unsafe Lines of Code with Vulnerability Types:\n")
for line, vuln_type in unsafe_lines:
    print(f"{line} - {vuln_type}")

test_inputs = tokenizer([cleaned_test_code], padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    test_outputs = model(**test_inputs)
test_embeddings = test_outputs.last_hidden_state.mean(dim=1).numpy()
prediction = clf.predict(test_embeddings)
# print("\nPrediction:", prediction)



Unsafe Lines of Code with Vulnerability Types:

eval(userInput); - Code Injection
document.getElementById('output').innerHTML = userInput; - Cross-Site Scripting (XSS)
document.write(userInput); - Cross-Site Scripting (XSS)


In [2]:
!pip install gradio


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 473.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 KB 256.9 kB/s eta 0:00:00 eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 KB 643.4 kB/s eta 0:00:001m641.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 KB 832.0 kB/s eta 0:00:0031m1.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 541.2 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 KB 323.4 kB/s eta 0:00:001m316.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 KB 267.7 kB/s eta 0:00:001m240.3 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.0 MB/s eta 0:00:001.2 MB/s eta 0:00:01
  Using cached urllib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 KB 691.9 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 971.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 789.5 kB/s eta 0:00:001m960.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 985.9 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 KB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 532.5 kB/s eta 0:00:001m503.9 kB/s eta 0:00:01
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5600 sha256=1c9ba021604e19e674b66f617808ead1585070d8abb37ee053c5175edd1473ae
  Stored in directory: /home/rohan/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [1]:
import gradio as gr

# Function to detect unsafe code
def detect_unsafe_code(code):
    # Clean and label the provided code
    cleaned_code = clean_code(code)
    unsafe_lines = label_code(cleaned_code)

    # Format the output
    formatted_output = "\n".join([f"{line[0]} - {line[1]}" for line in unsafe_lines])
    return formatted_output

# Create and launch the Gradio interface
iface = gr.Interface(
    fn=detect_unsafe_code,
    inputs="text",
    outputs="text",
    title="JavaScript Code Safety Analyzer",
    description="Enter your JavaScript code to detect unsafe lines."
)

iface.launch()


ModuleNotFoundError: No module named 'gradio'